# vCAT - Functional Localization


### Create feature space masks
#### Requires binarized FS masks for Fusiform and Parahippocampal
## ROI mask specs:
### Hippocampus (FS labels: hippocampus [17, 53])
### Putamen (FS labels: putamen [12, 51])
### Medial PFC (FS labels: rostral anterior cingulate [1026, 2026] & caudal anterior cingulate [1002, 2002])
### Dorsal anterior caudate (hand-drawn)


In [ ]:
#faces/fusiform

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks/ffa'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-006']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)


info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])

#get necessary files
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_faces_minus_scenes.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize fusiform mask using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [1007, 2007] #fusiform                         
anat_mask.inputs.binary_file = 'bi-fusiform.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS fusiform masks to EPI space (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize face zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize face zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

In [4]:
#faces/hippocampus

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks/hpc'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-006']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)


info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])

#get necessary files
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_faces_minus_scenes.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize fusiform mask using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [17, 53] #hippocampus                         
anat_mask.inputs.binary_file = 'bi-hippocampus.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS fusiform masks to EPI space (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize face zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize face zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features/hpc')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

220618-11:56:40,521 nipype.workflow INFO:
	 Workflow wf settings: ['check', 'execution', 'logging', 'monitoring']
220618-11:56:40,662 nipype.workflow INFO:
	 Running in parallel.
220618-11:56:40,666 nipype.workflow INFO:
	 Pending[0] Submitting[25] jobs Slots[inf]
220618-11:56:40,666 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 0
220618-11:56:40,919 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 0
220618-11:56:40,920 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 1
220618-11:56:41,106 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 1
220618-11:56:41,107 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 2
220618-11:56:41,274 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 2
220618-11:56:41,276 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 3
220618-11:56:41,444 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 3
220618-11:56:41,445 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 4
220618-11:56:41

220618-11:56:49,834 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 35
220618-11:56:49,835 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 36
220618-11:56:49,985 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 36
220618-11:56:49,986 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 37
220618-11:56:50,136 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 37
220618-11:56:50,137 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 38
220618-11:56:50,287 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 38
220618-11:56:50,288 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 39
220618-11:56:50,439 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 39
220618-11:56:50,440 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 40
220618-11:56:50,585 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 40
220618-11:56:50,587 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 41
220618-11:56:50,734 nipype.w

220618-11:56:58,645 nipype.workflow INFO:
	 [Job 44] Completed (wf.zstat_mask).
220618-11:56:58,924 nipype.workflow INFO:
	 [Job 46] Completed (wf.zstat_mask).
220618-11:56:59,212 nipype.workflow INFO:
	 [Job 48] Completed (wf.zstat_mask).
220618-11:56:59,498 nipype.workflow INFO:
	 [Job 50] Completed (wf.zstat_mask).
220618-11:56:59,780 nipype.workflow INFO:
	 [Job 52] Completed (wf.zstat_mask).
220618-11:57:00,63 nipype.workflow INFO:
	 [Job 54] Completed (wf.zstat_mask).
220618-11:57:00,349 nipype.workflow INFO:
	 [Job 56] Completed (wf.zstat_mask).
220618-11:57:00,635 nipype.workflow INFO:
	 [Job 58] Completed (wf.zstat_mask).
220618-11:57:00,784 nipype.workflow INFO:
	 [Job 59] Completed (wf.anat_mask).
220618-11:57:00,931 nipype.workflow INFO:
	 [Job 60] Completed (wf.zstat_mask).
220618-11:57:01,241 nipype.workflow INFO:
	 [Job 62] Completed (wf.zstat_mask).
220618-11:57:01,528 nipype.workflow INFO:
	 [Job 64] Completed (wf.zstat_mask).
220618-11:57:01,676 nipype.workflow INFO:


220618-11:57:11,284 nipype.workflow INFO:
	 [Job 95] Completed (wf.anatmask_xfm).
220618-11:57:14,285 nipype.workflow INFO:
	 Pending[22] Submitting[2] jobs Slots[inf]
220618-11:57:14,286 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 117
220618-11:57:14,444 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 117
220618-11:57:14,445 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 120
220618-11:57:14,592 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 120
220618-11:57:14,741 nipype.workflow INFO:
	 [Job 75] Completed (wf.anatmask_xfm).
220618-11:57:14,888 nipype.workflow INFO:
	 [Job 77] Completed (wf.anatmask_xfm).
220618-11:57:15,40 nipype.workflow INFO:
	 [Job 78] Completed (wf.anatmask_xfm).
220618-11:57:15,194 nipype.workflow INFO:
	 [Job 83] Completed (wf.anatmask_xfm).
220618-11:57:15,345 nipype.workflow INFO:
	 [Job 84] Completed (wf.anatmask_xfm).
220618-11:57:15,493 nipype.workflow INFO:
	 [Job 85] Completed (wf.anatmask_xfm).
220

220618-11:57:25,950 nipype.workflow INFO:
	 Submitting: wf.sink ID: 145
220618-11:57:26,96 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 145
220618-11:57:26,98 nipype.workflow INFO:
	 Submitting: wf.sink ID: 146
220618-11:57:26,259 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 146
220618-11:57:26,260 nipype.workflow INFO:
	 Submitting: wf.sink ID: 147
220618-11:57:26,440 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 147
220618-11:57:26,441 nipype.workflow INFO:
	 Submitting: wf.sink ID: 148
220618-11:57:26,603 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 148
220618-11:57:26,604 nipype.workflow INFO:
	 Submitting: wf.sink ID: 149
220618-11:57:26,755 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 149
220618-11:57:26,904 nipype.workflow INFO:
	 [Job 110] Completed (wf.feature_mask).
220618-11:57:27,55 nipype.workflow INFO:
	 [Job 111] Completed (wf.feature_mask).
220618-11:57:27,205 nipype.workflow INFO:
	 [Job 112] Completed (wf.feat

In [13]:
#faces/putamen

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks/hpc'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-006']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)


info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])

#get necessary files
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_faces_minus_scenes.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize putamen mask using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [12, 51] #putamen                         
anat_mask.inputs.binary_file = 'bi-putamen.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS putamen masks to EPI space (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize face zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize face zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features/put')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

220618-12:14:00,453 nipype.workflow INFO:
	 Workflow wf settings: ['check', 'execution', 'logging', 'monitoring']
220618-12:14:00,602 nipype.workflow INFO:
	 Running in parallel.
220618-12:14:00,606 nipype.workflow INFO:
	 Pending[0] Submitting[25] jobs Slots[inf]
220618-12:14:00,606 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 0
220618-12:14:01,37 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 0
220618-12:14:01,38 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 1
220618-12:14:01,233 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 1
220618-12:14:01,235 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 2
220618-12:14:01,434 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 2
220618-12:14:01,435 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 3
220618-12:14:01,632 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 3
220618-12:14:01,634 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 4
220618-12:14:01,8

220618-12:14:10,676 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 36
220618-12:14:10,677 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 37
220618-12:14:10,843 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 37
220618-12:14:10,844 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 38
220618-12:14:11,81 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 38
220618-12:14:11,82 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 39
220618-12:14:11,237 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 39
220618-12:14:11,238 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 40
220618-12:14:11,409 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 40
220618-12:14:11,411 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 41
220618-12:14:11,570 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 41
220618-12:14:11,571 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 42
220618-12:14:11,727 nipype.wor

220618-12:14:20,938 nipype.workflow INFO:
	 [Job 51] Completed (wf.anat_mask).
220618-12:14:21,94 nipype.workflow INFO:
	 [Job 52] Completed (wf.zstat_mask).
220618-12:14:21,244 nipype.workflow INFO:
	 [Job 53] Completed (wf.anat_mask).
220618-12:14:21,397 nipype.workflow INFO:
	 [Job 54] Completed (wf.zstat_mask).
220618-12:14:21,552 nipype.workflow INFO:
	 [Job 55] Completed (wf.anat_mask).
220618-12:14:21,701 nipype.workflow INFO:
	 [Job 56] Completed (wf.zstat_mask).
220618-12:14:21,855 nipype.workflow INFO:
	 [Job 57] Completed (wf.anat_mask).
220618-12:14:22,8 nipype.workflow INFO:
	 [Job 58] Completed (wf.zstat_mask).
220618-12:14:22,165 nipype.workflow INFO:
	 [Job 59] Completed (wf.anat_mask).
220618-12:14:22,315 nipype.workflow INFO:
	 [Job 60] Completed (wf.zstat_mask).
220618-12:14:22,464 nipype.workflow INFO:
	 [Job 61] Completed (wf.anat_mask).
220618-12:14:22,615 nipype.workflow INFO:
	 [Job 62] Completed (wf.zstat_mask).
220618-12:14:22,764 nipype.workflow INFO:
	 [Job 

220618-12:14:34,733 nipype.workflow INFO:
	 [Job 93] Completed (wf.anatmask_xfm).
220618-12:14:34,883 nipype.workflow INFO:
	 [Job 94] Completed (wf.anatmask_xfm).
220618-12:14:35,33 nipype.workflow INFO:
	 [Job 95] Completed (wf.anatmask_xfm).
220618-12:14:35,184 nipype.workflow INFO:
	 [Job 96] Completed (wf.anatmask_xfm).
220618-12:14:35,336 nipype.workflow INFO:
	 [Job 97] Completed (wf.anatmask_xfm).
220618-12:14:35,490 nipype.workflow INFO:
	 [Job 98] Completed (wf.anatmask_xfm).
220618-12:14:35,641 nipype.workflow INFO:
	 [Job 99] Completed (wf.anatmask_xfm).
220618-12:14:35,795 nipype.workflow INFO:
	 [Job 75] Completed (wf.anatmask_xfm).
220618-12:14:35,946 nipype.workflow INFO:
	 [Job 102] Completed (wf.feature_mask).
220618-12:14:36,97 nipype.workflow INFO:
	 [Job 103] Completed (wf.feature_mask).
220618-12:14:36,237 nipype.workflow INFO:
	 Pending[1] Submitting[24] jobs Slots[inf]
220618-12:14:36,238 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 100
220618-12:14:3

220618-12:14:45,220 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 135
220618-12:14:45,221 nipype.workflow INFO:
	 Submitting: wf.sink ID: 136
220618-12:14:45,372 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 136
220618-12:14:45,373 nipype.workflow INFO:
	 Submitting: wf.sink ID: 137
220618-12:14:45,525 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 137
220618-12:14:45,526 nipype.workflow INFO:
	 Submitting: wf.sink ID: 138
220618-12:14:45,675 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 138
220618-12:14:45,676 nipype.workflow INFO:
	 Submitting: wf.sink ID: 139
220618-12:14:45,826 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 139
220618-12:14:45,829 nipype.workflow INFO:
	 Submitting: wf.sink ID: 140
220618-12:14:45,981 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 140
220618-12:14:45,982 nipype.workflow INFO:
	 Submitting: wf.sink ID: 143
220618-12:14:46,135 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 143
2

In [8]:
#scenes/parahippocampal

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks/ppa'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-006', 'sub-007']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)

#get necessary files
info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_scenes_minus_faces.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize parahippocampal masks using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [1016, 2016] #parahippocampal                         
anat_mask.inputs.binary_file = 'bi-parahippocampal.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS parahippocampal masks to EPI space (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize scene zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize scene zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

220618-12:06:07,991 nipype.workflow INFO:
	 Workflow wf settings: ['check', 'execution', 'logging', 'monitoring']
220618-12:06:08,140 nipype.workflow INFO:
	 Running in parallel.
220618-12:06:08,144 nipype.workflow INFO:
	 Pending[0] Submitting[25] jobs Slots[inf]
220618-12:06:08,145 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 0
220618-12:06:08,423 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 0
220618-12:06:08,424 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 1
220618-12:06:08,646 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 1
220618-12:06:08,648 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 2
220618-12:06:08,839 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 2
220618-12:06:08,840 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 3
220618-12:06:09,34 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 3
220618-12:06:09,35 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 4
220618-12:06:09,2

220618-12:06:18,619 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 36
220618-12:06:18,620 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 37
220618-12:06:18,801 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 37
220618-12:06:18,802 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 38
220618-12:06:18,961 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 38
220618-12:06:18,962 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 39
220618-12:06:19,114 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 39
220618-12:06:19,115 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 40
220618-12:06:19,287 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 40
220618-12:06:19,288 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 41
220618-12:06:19,459 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 41
220618-12:06:19,460 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 42
220618-12:06:19,614 nipype.w

220618-12:06:28,945 nipype.workflow INFO:
	 [Job 51] Completed (wf.anat_mask).
220618-12:06:29,107 nipype.workflow INFO:
	 [Job 52] Completed (wf.zstat_mask).
220618-12:06:29,257 nipype.workflow INFO:
	 [Job 53] Completed (wf.anat_mask).
220618-12:06:29,416 nipype.workflow INFO:
	 [Job 54] Completed (wf.zstat_mask).
220618-12:06:29,567 nipype.workflow INFO:
	 [Job 55] Completed (wf.anat_mask).
220618-12:06:29,719 nipype.workflow INFO:
	 [Job 56] Completed (wf.zstat_mask).
220618-12:06:29,871 nipype.workflow INFO:
	 [Job 57] Completed (wf.anat_mask).
220618-12:06:30,24 nipype.workflow INFO:
	 [Job 58] Completed (wf.zstat_mask).
220618-12:06:30,175 nipype.workflow INFO:
	 [Job 59] Completed (wf.anat_mask).
220618-12:06:30,330 nipype.workflow INFO:
	 [Job 60] Completed (wf.zstat_mask).
220618-12:06:30,479 nipype.workflow INFO:
	 [Job 61] Completed (wf.anat_mask).
220618-12:06:30,633 nipype.workflow INFO:
	 [Job 62] Completed (wf.zstat_mask).
220618-12:06:30,783 nipype.workflow INFO:
	 [Jo

220618-12:06:43,115 nipype.workflow INFO:
	 [Job 95] Completed (wf.anatmask_xfm).
220618-12:06:43,274 nipype.workflow INFO:
	 [Job 96] Completed (wf.anatmask_xfm).
220618-12:06:43,431 nipype.workflow INFO:
	 [Job 97] Completed (wf.anatmask_xfm).
220618-12:06:43,589 nipype.workflow INFO:
	 [Job 98] Completed (wf.anatmask_xfm).
220618-12:06:43,742 nipype.workflow INFO:
	 [Job 99] Completed (wf.anatmask_xfm).
220618-12:06:43,897 nipype.workflow INFO:
	 [Job 80] Completed (wf.anatmask_xfm).
220618-12:06:44,59 nipype.workflow INFO:
	 [Job 103] Completed (wf.feature_mask).
220618-12:06:44,216 nipype.workflow INFO:
	 [Job 104] Completed (wf.feature_mask).
220618-12:06:44,219 nipype.workflow INFO:
	 Pending[0] Submitting[25] jobs Slots[inf]
220618-12:06:44,220 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 100
220618-12:06:44,373 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 100
220618-12:06:44,374 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 101
220618-1

220618-12:06:55,740 nipype.workflow INFO:
	 [Job 129] Completed (wf.sink).
220618-12:06:55,892 nipype.workflow INFO:
	 [Job 125] Completed (wf.sink).
220618-12:06:56,51 nipype.workflow INFO:
	 [Job 143] Completed (wf.sink).
220618-12:06:56,205 nipype.workflow INFO:
	 [Job 144] Completed (wf.sink).
220618-12:06:56,360 nipype.workflow INFO:
	 [Job 145] Completed (wf.sink).
220618-12:06:56,510 nipype.workflow INFO:
	 [Job 146] Completed (wf.sink).
220618-12:06:56,660 nipype.workflow INFO:
	 [Job 147] Completed (wf.sink).
220618-12:06:56,813 nipype.workflow INFO:
	 [Job 148] Completed (wf.sink).
220618-12:06:56,963 nipype.workflow INFO:
	 [Job 149] Completed (wf.sink).
220618-12:06:56,966 nipype.workflow INFO:
	 Pending[0] Submitting[15] jobs Slots[inf]
220618-12:06:56,967 nipype.workflow INFO:
	 Submitting: wf.sink ID: 126
220618-12:06:57,131 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 126
220618-12:06:57,132 nipype.workflow INFO:
	 Submitting: wf.sink ID: 127
220618-12:06:57

In [ ]:
#scenes/hippocampus

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks/hpc-scenes'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-006', 'sub-007']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)

#get necessary files
info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_scenes_minus_faces.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize hippocampal mask using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [17, 53] #hippocampus                         
anat_mask.inputs.binary_file = 'bi-hippocampus.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS hippocampal masks to EPI space (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize scene zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize scene zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features/hpc')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

In [15]:
#scenes/putamen

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks/hpc-scenes'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-006', 'sub-007']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)

#get necessary files
info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_scenes_minus_faces.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize putamen mask using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [12, 51] #hippocampus                         
anat_mask.inputs.binary_file = 'bi-putamen.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS putamen masks to EPI space (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize scene zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize scene zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features/put')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

220618-12:18:37,852 nipype.workflow INFO:
	 Workflow wf settings: ['check', 'execution', 'logging', 'monitoring']
220618-12:18:38,29 nipype.workflow INFO:
	 Running in parallel.
220618-12:18:38,34 nipype.workflow INFO:
	 Pending[0] Submitting[25] jobs Slots[inf]
220618-12:18:38,34 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 0
220618-12:18:38,335 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 0
220618-12:18:38,337 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 1
220618-12:18:38,558 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 1
220618-12:18:38,560 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 2
220618-12:18:38,770 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 2
220618-12:18:38,771 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 3
220618-12:18:38,994 nipype.workflow INFO:
	 Finished submitting: wf.datasource ID: 3
220618-12:18:38,995 nipype.workflow INFO:
	 Submitting: wf.datasource ID: 4
220618-12:18:39,19

220618-12:18:46,755 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 32
220618-12:18:46,756 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 33
220618-12:18:46,761 nipype.workflow INFO:
	 [Job 33] Cached (wf.anat_mask).
220618-12:18:46,762 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 33
220618-12:18:46,763 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 34
220618-12:18:46,767 nipype.workflow INFO:
	 [Job 34] Cached (wf.zstat_mask).
220618-12:18:46,768 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 34
220618-12:18:46,769 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 35
220618-12:18:46,774 nipype.workflow INFO:
	 [Job 35] Cached (wf.anat_mask).
220618-12:18:46,775 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 35
220618-12:18:46,776 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 36
220618-12:18:46,780 nipype.workflow INFO:
	 [Job 36] Cached (wf.zstat_mask).
220618-12:18:46,782 nipype.workflow INFO:
	 Fi

220618-12:18:46,979 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 67
220618-12:18:46,983 nipype.workflow INFO:
	 [Job 67] Cached (wf.anat_mask).
220618-12:18:46,984 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 67
220618-12:18:46,985 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 68
220618-12:18:46,990 nipype.workflow INFO:
	 [Job 68] Cached (wf.zstat_mask).
220618-12:18:46,991 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 68
220618-12:18:46,992 nipype.workflow INFO:
	 Submitting: wf.anat_mask ID: 69
220618-12:18:46,999 nipype.workflow INFO:
	 [Job 69] Cached (wf.anat_mask).
220618-12:18:47,0 nipype.workflow INFO:
	 Finished submitting: wf.anat_mask ID: 69
220618-12:18:47,1 nipype.workflow INFO:
	 Submitting: wf.zstat_mask ID: 70
220618-12:18:47,9 nipype.workflow INFO:
	 [Job 70] Cached (wf.zstat_mask).
220618-12:18:47,10 nipype.workflow INFO:
	 Finished submitting: wf.zstat_mask ID: 70
220618-12:18:47,11 nipype.workflow INFO:
	 Submitting

220618-12:18:47,316 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 100
220618-12:18:47,323 nipype.workflow INFO:
	 [Job 100] Cached (wf.feature_mask).
220618-12:18:47,325 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 100
220618-12:18:47,325 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 101
220618-12:18:47,333 nipype.workflow INFO:
	 [Job 101] Cached (wf.feature_mask).
220618-12:18:47,335 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 101
220618-12:18:47,335 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 102
220618-12:18:47,342 nipype.workflow INFO:
	 [Job 102] Cached (wf.feature_mask).
220618-12:18:47,344 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 102
220618-12:18:47,345 nipype.workflow INFO:
	 Submitting: wf.feature_mask ID: 103
220618-12:18:47,352 nipype.workflow INFO:
	 [Job 103] Cached (wf.feature_mask).
220618-12:18:47,354 nipype.workflow INFO:
	 Finished submitting: wf.feature_mask ID: 103
2206

220618-12:18:49,76 nipype.workflow INFO:
	 Submitting: wf.sink ID: 133
220618-12:18:49,81 nipype.workflow INFO:
	 [Node] Outdated cache found for "wf.sink".
220618-12:18:49,242 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 133
220618-12:18:49,243 nipype.workflow INFO:
	 Submitting: wf.sink ID: 134
220618-12:18:49,250 nipype.workflow INFO:
	 [Node] Outdated cache found for "wf.sink".
220618-12:18:49,408 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 134
220618-12:18:49,409 nipype.workflow INFO:
	 Submitting: wf.sink ID: 135
220618-12:18:49,415 nipype.workflow INFO:
	 [Node] Outdated cache found for "wf.sink".
220618-12:18:49,570 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 135
220618-12:18:49,571 nipype.workflow INFO:
	 Submitting: wf.sink ID: 136
220618-12:18:49,578 nipype.workflow INFO:
	 [Node] Outdated cache found for "wf.sink".
220618-12:18:49,738 nipype.workflow INFO:
	 Finished submitting: wf.sink ID: 136
220618-12:18:49,739 nipype.workflow INFO:


#### Move shit

In [16]:
import os
from os.path import join, basename
from glob import glob
import shutil 

sids = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
sids1 = ['sub-005']
for sid in sids:
    mask_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/loc-anat_masks'
    
    if not os.path.exists(join(mask_dir, '{0}/'.format(sid))):
        os.makedirs(join(mask_dir, '{0}/'.format(sid)))

    shutil.move(join(mask_dir, 'features/put/_subject_id_{0}/zstat_faces_minus_scenes_thresh_maths.nii.gz'.format(sid)),
                join(mask_dir, '{0}/feature-put_faces.nii.gz'.format(sid)))
    
    shutil.move(join(mask_dir, 'features/put/_subject_id_{0}/zstat_scenes_minus_faces_thresh_maths.nii.gz'.format(sid)),
                join(mask_dir, '{0}/feature-put_scenes.nii.gz'.format(sid)))              